In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from textblob import TextBlob
import re

from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *

In [12]:
tweetData = pd.read_csv('Test29-02.csv',usecols=['Date','Tweets'])
stockData = yf.download("AAPL", start="2020-06-29", end="2020-07-03")

stockData = stockData.drop(columns=['Open','High','Low','Close','Volume'],axis=1)
stockData = stockData.rename(columns={'Adj Close':'AdjClose'})
stockData = stockData.reset_index()
stockData['Date'] = stockData['Date'].astype(str)
stockData.set_index('Date',inplace=True)

[*********************100%***********************]  1 of 1 completed


In [13]:
#stockData.drop(['2019-12-31'],inplace=True)
tweetData

,Date,Tweets
0,2020-06-29,ba backtesting upwards trendline it can possi...
1,2020-06-30,here is my semiannual report for the past mon...
2,2020-07-01,follow for optionsignals optiontrading will go...
3,2020-07-02,msft outperforming aapl fb nflf today ready f...


In [30]:
def cleanTweets(tweets):
    tweets = re.sub('[^a-zA-Z\s]*', '', tweets)
    return tweets

In [31]:
tweetData['Tweets'] = tweetData['Tweets'].apply(cleanTweets)

In [14]:
stockData.tail()

,AdjClose
Date,
2020-06-29,361.779999
2020-06-30,364.799988
2020-07-01,364.109985
2020-07-02,366.630005


In [15]:
mergedData = pd.DataFrame(tweetData,columns=['Date','Tweets','AdjClose'])
mergedData.head()

,Date,Tweets,AdjClose
0,2020-06-29,ba backtesting upwards trendline it can possi...,NaN
1,2020-06-30,here is my semiannual report for the past mon...,NaN
2,2020-07-01,follow for optionsignals optiontrading will go...,NaN
3,2020-07-02,msft outperforming aapl fb nflf today ready f...,NaN


In [16]:
mergedData.at[0,'AdjClose'] = 361.779999
mergedData.at[1,'AdjClose'] = 364.799988
mergedData.at[2,'AdjClose'] = 364.109985
mergedData.at[3,'AdjClose'] = 366.519989

In [17]:
mergedData['Target'] = ''

for i in range(3):
    mergedData.at[i,'Target'] = mergedData.at[i+1,'AdjClose']
    
mergedData.drop([3],inplace=True)
mergedData

,Date,Tweets,AdjClose,Target
0,2020-06-29,ba backtesting upwards trendline it can possi...,361.779999,364.8
1,2020-06-30,here is my semiannual report for the past mon...,364.799988,364.11
2,2020-07-01,follow for optionsignals optiontrading will go...,364.109985,366.52


In [11]:
mergedData.set_index(['Date'],inplace=True)

In [12]:
for date in list(stockData.index):
    mergedData.at[date,'AdjClose'] = stockData.at[date,'AdjClose']

In [11]:
mergedData['AdjClose'].isnull().sum()

56

In [12]:
#mergedData.to_csv('FINALDATASET.csv')

## REMOVING ROWS CONTAINING NaN

In [13]:
mergedData=mergedData.dropna()

# Sentiment Analysis

In [18]:
import nltk
nltk.download('vader_lexicon')

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import unicodedata

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Rahul\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [19]:
analyzer = SentimentIntensityAnalyzer()

for i,tweet in zip(mergedData.index,mergedData.Tweets):
    #sentence_i = unicodedata.normalize('NFKD', tweet)
    sentence_sentiment = analyzer.polarity_scores(str(tweet))
    mergedData.at[i,'Comp'] = sentence_sentiment['compound']
    mergedData.at[i,'Pos'] = sentence_sentiment['pos']
    mergedData.at[i,'Neg'] = sentence_sentiment['neg']
    mergedData.at[i,'Neu'] = sentence_sentiment['neu']

In [44]:
#mergedData.to_csv('test1-Jul.csv')

In [20]:
mergedData

,Date,Tweets,AdjClose,Target,Comp,Pos,Neg,Neu
0,2020-06-29,ba backtesting upwards trendline it can possi...,361.779999,364.8,0.9999,0.116,0.032,0.852
1,2020-06-30,here is my semiannual report for the past mon...,364.799988,364.11,0.9999,0.110,0.027,0.864
2,2020-07-01,follow for optionsignals optiontrading will go...,364.109985,366.52,0.9999,0.110,0.026,0.864


In [17]:
string = 'It was not very good'
sentence_sentiment = analyzer.polarity_scores(string)
print(sentence_sentiment)

sentiments = TextBlob(string)
print(sentiments.sentiment)

{'neg': 0.396, 'neu': 0.604, 'pos': 0.0, 'compound': -0.3865}
Sentiment(polarity=-0.26923076923076916, subjectivity=0.46153846153846156)


In [18]:
for i,tweet in zip(mergedData.index,mergedData.Tweets):
    sentiments = TextBlob(str(tweet))
    mergedData.at[i,'TextBlobPolarity'] = sentiments.sentiment.polarity
    
mergedData

,Tweets,AdjClose,Comp,Pos,Neg,Neu,TextBlobPolarity
Date,,,,,,,
2020-01-02,is already a memetic year tsla is over curren...,298.829956,0.9992,0.145,0.063,0.791,0.107992
2020-01-03,the sectorselector for is posted technology x...,295.924713,0.9987,0.107,0.061,0.832,0.121075
2020-01-06,if youre buying this stock now youre doing it ...,298.282715,0.9964,0.086,0.049,0.865,0.119084
2020-01-07,if you invested in snapchat a year ago you wou...,296.879883,0.9311,0.073,0.060,0.867,0.128024
2020-01-08,view todays marketoutlook from discussed spy...,301.655548,0.9945,0.092,0.061,0.847,0.077918
...,...,...,...,...,...,...,...
2020-06-19,president trump bringing back the gold standar...,349.720001,0.9996,0.113,0.037,0.849,0.163768
2020-06-22,top increases in users holding for stocks held...,358.869995,0.9997,0.118,0.036,0.846,0.224847
2020-06-23,aapl apple appeared to have brokeout but weak ...,366.529999,0.9990,0.087,0.042,0.870,0.129869


In [26]:
mergedData=mergedData.drop(['Tweets'],axis=1)

In [27]:
mergedData = mergedData.reset_index()
mergedData.to_csv('train.csv')

In [20]:
Y = np.array(mergedData.AdjClose[1:])
Y = Y.reshape(120,1)

In [21]:
Y.shape

(120, 1)

In [22]:
X = pd.DataFrame(mergedData,columns=['AdjClose','Comp','Pos','Neg','Neu'])
X = X[:120].to_numpy()
X.shape

(120, 5)

In [23]:
xtrain, xtest, ytrain, ytest=train_test_split(X, Y, test_size=0.2)

NameError: name 'train_test_split' is not defined